In [121]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
import cv2
import os


# TP2

In [136]:

def load_detections(fichier_det):
    # Noms des colonnes selon le format MOT Challenge
    noms_colonnes = ['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y', 'z']
    
    # Charger le fichier en utilisant pandas
    detections = pd.read_csv(fichier_det, header=None, names=noms_colonnes, sep=' ')
    
    return detections

# Charger les détections depuis les différents modèles
det_yolov5s = load_detections('ADL-Rundle-6/det/Yolov5s/det.txt')
det_yolov5l = load_detections('ADL-Rundle-6/det/Yolov5l/det.txt')

# Afficher les premières lignes pour vérification
print("Détections YOLOv5s:")
print(det_yolov5s.head())
print("\nDétections YOLOv5l:")
print(det_yolov5l.head())


Détections YOLOv5s:
   frame  id  bb_left  bb_top  bb_width  bb_height      conf  x  y  z
0      1  -1     1695     386       136        342  0.760947 -1 -1 -1
1      1  -1      255     459       103        246  0.676300 -1 -1 -1
2      1  -1     1885     380        34        202  0.628092 -1 -1 -1
3      1  -1        2     306        81        614  0.575901 -1 -1 -1
4      1  -1     1284     463        74        195  0.575504 -1 -1 -1

Détections YOLOv5l:
   frame  id  bb_left  bb_top  bb_width  bb_height      conf  x  y  z
0      1  -1     1700     391       156        337  0.914550 -1 -1 -1
1      1  -1      250     456       107        248  0.883148 -1 -1 -1
2      1  -1     1255     539        60        118  0.826354 -1 -1 -1
3      1  -1     1288     459        73        199  0.745969 -1 -1 -1
4      1  -1      120     504        93        239  0.740778 -1 -1 -1


In [137]:
def calculate_iou(box1, box2):
    b1_x1, b1_y1 = box1[0], box1[1]
    b1_x2, b1_y2 = box1[0] + box1[2], box1[1] + box1[3]
    b2_x1, b2_y1 = box2[0], box2[1] 
    b2_x2, b2_y2 = box2[0] + box2[2], box2[1] + box2[3]

    inter_x1 = max(b1_x1, b2_x1)
    inter_y1 = max(b1_y1, b2_y1)
    inter_x2 = min(b1_x2, b2_x2)
    inter_y2 = min(b1_y2, b2_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    
    union_area = b1_area + b2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

In [138]:
def calculate_similarity_matrix(tracked_boxes, new_detections):
    # Vérifier si les entrées sont des DataFrames ou des listes
    if hasattr(tracked_boxes, 'iloc'):
        tracked_bboxes = tracked_boxes.iloc[:, 2:6].values.tolist()
    else:
        tracked_bboxes = tracked_boxes
        
    if hasattr(new_detections, 'iloc'):
        detection_bboxes = new_detections.iloc[:, 2:6].values.tolist()
    else:
        detection_bboxes = new_detections
    
    # Initialiser la matrice de similarité
    n_tracked = len(tracked_bboxes)
    n_detections = len(detection_bboxes)
    similarity_matrix = np.zeros((n_tracked, n_detections))
    
    # Calculer l'IoU pour chaque paire de boîtes
    for i in range(n_tracked):
        for j in range(n_detections):
            similarity_matrix[i,j] = calculate_iou(tracked_bboxes[i], detection_bboxes[j])
            
    return similarity_matrix, tracked_bboxes, detection_bboxes


In [139]:
similarity_matrix, tracked_bboxes, detection_bboxes = calculate_similarity_matrix(det_yolov5s, det_yolov5l)
print("Matrice de similarité:")
print(similarity_matrix)
print("\nBoîtes suivies:")
print(tracked_bboxes)
print("\nNouvelles détections:")
print(detection_bboxes)

Matrice de similarité:
[[0.80359321 0.         0.         ... 0.         0.15574894 0.        ]
 [0.         0.92954917 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.13648294 ... 0.         0.         0.70013348]
 [0.         0.         0.         ... 0.         0.         0.06436677]
 [0.         0.         0.         ... 0.         0.         0.        ]]

Boîtes suivies:
[[1695, 386, 136, 342], [255, 459, 103, 246], [1885, 380, 34, 202], [2, 306, 81, 614], [1284, 463, 74, 195], [259, 456, 106, 247], [1694, 387, 126, 341], [1884, 376, 35, 215], [2, 309, 86, 603], [1255, 532, 53, 126], [120, 510, 69, 236], [1287, 465, 80, 192], [1332, 488, 47, 158], [263, 457, 99, 245], [1691, 390, 120, 340], [1883, 376, 35, 215], [0, 313, 117, 590], [115, 509, 74, 237], [1257, 535, 50, 121], [1288, 462, 79, 194], [1725, 399, 143, 335], [1331, 485, 49, 162], [259, 457, 108, 248], [1, 299, 113, 59

In [140]:
cost_matrix = 1 - similarity_matrix
tracked_indices, detection_indices = linear_sum_assignment(cost_matrix)

print("Assignations optimales:")
for track_idx, det_idx in zip(tracked_indices, detection_indices):
    print(f"Track {track_idx} -> Detection {det_idx} (IoU: {similarity_matrix[track_idx, det_idx]:.3f})")


Assignations optimales:
Track 0 -> Detection 7 (IoU: 0.826)
Track 1 -> Detection 1 (IoU: 0.930)
Track 2 -> Detection 6 (IoU: 0.921)
Track 3 -> Detection 1305 (IoU: 0.791)
Track 5 -> Detection 378 (IoU: 0.303)
Track 7 -> Detection 13 (IoU: 0.914)
Track 8 -> Detection 1558 (IoU: 0.603)
Track 9 -> Detection 2 (IoU: 0.833)
Track 10 -> Detection 591 (IoU: 0.952)
Track 13 -> Detection 37 (IoU: 0.945)
Track 15 -> Detection 20 (IoU: 0.843)
Track 16 -> Detection 1023 (IoU: 0.922)
Track 17 -> Detection 658 (IoU: 0.941)
Track 18 -> Detection 9 (IoU: 0.816)
Track 19 -> Detection 3 (IoU: 0.903)
Track 22 -> Detection 386 (IoU: 0.297)
Track 25 -> Detection 27 (IoU: 0.847)
Track 26 -> Detection 669 (IoU: 0.937)
Track 29 -> Detection 17 (IoU: 0.819)
Track 31 -> Detection 8 (IoU: 0.932)
Track 32 -> Detection 23 (IoU: 0.947)
Track 33 -> Detection 909 (IoU: 0.925)
Track 34 -> Detection 34 (IoU: 0.835)
Track 36 -> Detection 39 (IoU: 0.837)
Track 38 -> Detection 1427 (IoU: 0.537)
Track 39 -> Detection 22 (I

In [141]:
MAX_FRAMES_PERDUES = 20  # Nombre maximum de frames où une trajectoire peut être perdue
SEUIL_IOU = 0.1
matches = []  # Paires (track_id, detection_id) associées
unmatched_tracks = []  # Trajectoires non associées
unmatched_detections = []  # Détections non associées

for track_idx, det_idx in zip(tracked_indices, detection_indices):
    if similarity_matrix[track_idx, det_idx] >= SEUIL_IOU:
        matches.append((track_idx, det_idx))
    else:
        unmatched_tracks.append(track_idx)
        unmatched_detections.append(det_idx)

# Ajouter les indices non utilisés aux listes non appariées
all_tracks = set(range(len(tracked_bboxes)))
all_detections = set(range(len(detection_bboxes)))

matched_tracks = set(match[0] for match in matches)
matched_detections = set(match[1] for match in matches)

unmatched_tracks.extend(list(all_tracks - matched_tracks))
unmatched_detections.extend(list(all_detections - matched_detections))

print("\nRésultats de l'association:")
print(f"Nombre d'associations: {len(matches)}")
print(f"Trajectoires non associées: {len(unmatched_tracks)}")
print(f"Détections non associées: {len(unmatched_detections)}")


print("\nMise à jour des trajectoires:")
for track_idx, det_idx in matches:
    print(f"Trajectoire {track_idx} mise à jour avec détection {det_idx}")
    

print("\nTrajectoires non associées à supprimer si dépassement du seuil:")
for track_idx in unmatched_tracks:
    print(f"Trajectoire {track_idx} non associée")


print("\nNouvelles trajectoires à créer:")
for det_idx in unmatched_detections:
    print(f"Nouvelle trajectoire pour détection {det_idx}")


Résultats de l'association:
Nombre d'associations: 4470
Trajectoires non associées: 3174
Détections non associées: 184

Mise à jour des trajectoires:
Trajectoire 0 mise à jour avec détection 7
Trajectoire 1 mise à jour avec détection 1
Trajectoire 2 mise à jour avec détection 6
Trajectoire 3 mise à jour avec détection 1305
Trajectoire 5 mise à jour avec détection 378
Trajectoire 7 mise à jour avec détection 13
Trajectoire 8 mise à jour avec détection 1558
Trajectoire 9 mise à jour avec détection 2
Trajectoire 10 mise à jour avec détection 591
Trajectoire 13 mise à jour avec détection 37
Trajectoire 15 mise à jour avec détection 20
Trajectoire 16 mise à jour avec détection 1023
Trajectoire 17 mise à jour avec détection 658
Trajectoire 18 mise à jour avec détection 9
Trajectoire 19 mise à jour avec détection 3
Trajectoire 22 mise à jour avec détection 386
Trajectoire 25 mise à jour avec détection 27
Trajectoire 26 mise à jour avec détection 669
Trajectoire 29 mise à jour avec détection 

In [142]:
def visualise_tracking_results(video_path, tracked_objects, output_path):
    seq_info = {}
    with open(os.path.join(video_path, 'seqinfo.ini')) as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=')
                seq_info[key] = value

    # Configuration de la sortie vidéo
    frame_width = int(seq_info['imWidth'])
    frame_height = int(seq_info['imHeight'])
    fps = int(seq_info['frameRate'])
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Dictionnaire pour stocker les objets par frame
    objects_by_frame = {}
    
    # Réorganiser les objets par frame
    for track_id, track in tracked_objects.items():
        for frame_idx, bbox in track.items():
            if frame_idx not in objects_by_frame:
                objects_by_frame[frame_idx] = []
            objects_by_frame[frame_idx].append((track_id, bbox))


    # Pour chaque frame
    for frame_idx in range(1, int(seq_info['seqLength']) + 1):
        # Lecture de l'image
        img_path = os.path.join(video_path, 'img1', f'{frame_idx:06d}.jpg')
        frame = cv2.imread(img_path)
        
        if frame is None:
            continue

        # Dessiner tous les objets présents dans cette frame
        if frame_idx in objects_by_frame:
            for track_id, bbox in objects_by_frame[frame_idx]:
                x, y, w, h = map(int, bbox)
                
                # Dessin de la boîte englobante
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Affichage de l'ID
                cv2.putText(frame, f'ID: {track_id}', 
                          (x, y - 10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 
                          0.9, 
                          (0, 255, 0), 
                          2)

        # Affichage de la frame
        cv2.imshow('Tracking Results', frame)
        
        # Sauvegarde de la frame dans la vidéo
        out.write(frame)
        
        # Attente de 30ms, sortie si 'q' est pressé
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    # Nettoyage
    out.release()
    cv2.destroyAllWindows()

# Exemple d'utilisation:
video_path = 'ADL-Rundle-6'
output_path = 'tp2_result.mp4'
# Créer un dictionnaire pour stocker les objets suivis
tracked_objects = {}


In [143]:
class Track:
    def __init__(self, bbox, track_id):
        self.id = track_id
        self.bbox = bbox
        self.frames_perdus = 0
    
    def predict(self):
        # Sans Kalman, on garde simplement la dernière position connue
        return self.bbox
    
    def update(self, detection):
        # Mise à jour directe avec la nouvelle détection
        self.bbox = detection
        self.frames_perdus = 0

In [144]:
def tracker(detections_par_frame):
    tracks = []
    next_track_id = 1
    resultats = {}
    active_tracks = set()  # Pour suivre les IDs actifs
    
    for frame_id, detections in detections_par_frame.items():
        # 1. Prédiction pour toutes les trajectoires existantes
        for track in tracks:
            track.predict()
        
        # 2. Association avec les nouvelles détections
        if tracks and len(detections) > 0:
            similarity_matrix = np.zeros((len(tracks), len(detections)))
            for i, track in enumerate(tracks):
                for j, detection in enumerate(detections):
                    # Vérifier si les boîtes se chevauchent significativement
                    iou = calculate_iou(track.bbox, detection)
                    similarity_matrix[i,j] = iou
            
            # Augmenter le seuil IoU pour éviter les associations ambiguës
            SEUIL_IOU = 0.5  # Seuil plus strict
            similarity_matrix[similarity_matrix < SEUIL_IOU] = 0
            
            # Ne permettre qu'une seule association par détection
            cost_matrix = 1 - similarity_matrix
            track_indices, det_indices = linear_sum_assignment(cost_matrix)
            
            matches = []
            unmatched_tracks = list(range(len(tracks)))
            unmatched_detections = list(range(len(detections)))
            
            # Filtrer les associations en vérifiant les chevauchements
            for track_idx, det_idx in zip(track_indices, det_indices):
                if similarity_matrix[track_idx, det_idx] >= SEUIL_IOU:
                    # Vérifier s'il n'y a pas déjà une trajectoire active dans cette zone
                    overlap_found = False
                    for other_track in tracks:
                        if other_track.id != tracks[track_idx].id and \
                           other_track.frames_perdus == 0 and \
                           calculate_iou(other_track.bbox, detections[det_idx]) > 0.3:
                            overlap_found = True
                            break
                    
                    if not overlap_found:
                        matches.append((track_idx, det_idx))
                        if track_idx in unmatched_tracks:
                            unmatched_tracks.remove(track_idx)
                        if det_idx in unmatched_detections:
                            unmatched_detections.remove(det_idx)
        else:
            matches = []
            unmatched_tracks = list(range(len(tracks)))
            unmatched_detections = list(range(len(detections)))
        
        # 3. Mise à jour des trajectoires
        for track_idx, det_idx in matches:
            tracks[track_idx].update(detections[det_idx])
            resultats.setdefault(tracks[track_idx].id, {})[frame_id] = tracks[track_idx].bbox
            active_tracks.add(tracks[track_idx].id)
        
        # 4. Gestion des trajectoires non associées
        MAX_FRAMES_PERDUES = 15  # Réduire pour supprimer plus rapidement les trajectoires perdues
        for track_idx in unmatched_tracks:
            tracks[track_idx].frames_perdus += 1
            if tracks[track_idx].frames_perdus <= MAX_FRAMES_PERDUES:
                resultats.setdefault(tracks[track_idx].id, {})[frame_id] = tracks[track_idx].bbox
            else:
                active_tracks.discard(tracks[track_idx].id)
        
        # 5. Supprimer les trajectoires perdues
        tracks = [track for track in tracks if track.frames_perdus <= MAX_FRAMES_PERDUES]
        
        # 6. Créer de nouvelles trajectoires
        MIN_DISTANCE = 50  # Distance minimale entre les centres des boîtes
        for det_idx in unmatched_detections:
            # Vérifier si la détection est suffisamment éloignée des trajectoires existantes
            detection = detections[det_idx]
            det_center = (detection[0] + detection[2]/2, detection[1] + detection[3]/2)
            
            trop_proche = False
            for track in tracks:
                track_box = track.bbox
                track_center = (track_box[0] + track_box[2]/2, track_box[1] + track_box[3]/2)
                distance = np.sqrt((det_center[0] - track_center[0])**2 + 
                                 (det_center[1] - track_center[1])**2)
                if distance < MIN_DISTANCE:
                    trop_proche = True
                    break
            
            if not trop_proche:
                new_track = Track(detection, next_track_id)
                tracks.append(new_track)
                resultats.setdefault(next_track_id, {})[frame_id] = detection
                active_tracks.add(next_track_id)
                next_track_id += 1
    
    # 7. Filtrer les trajectoires trop courtes
    MIN_DETECTIONS = 5  # Augmenter pour avoir des trajectoires plus stables
    trajectoires_filtrees = {}
    for track_id, track_data in resultats.items():
        if len(track_data) >= MIN_DETECTIONS:
            trajectoires_filtrees[track_id] = track_data
    
    return trajectoires_filtrees

In [145]:
# Organiser les détections par frame
detections_yolov5s_par_frame = {
    frame: det_yolov5s[det_yolov5s['frame'] == frame].iloc[:, 2:6].values.tolist()
    for frame in det_yolov5s['frame'].unique()
}

# Lancer le tracking
resultats_tracking = tracker(detections_yolov5s_par_frame)

# Visualiser les résultats
visualise_tracking_results('ADL-Rundle-6', resultats_tracking, output_path)

In [17]:
def sauvegarder_resultats(tracked_objects, sequence_name):
    # Créer le fichier de sortie avec le même nom que la séquence
    output_file = f'{sequence_name}.txt'
    
    with open(output_file, 'w') as f:
        # Pour chaque objet suivi
        for track_id in tracked_objects:
            # Pour chaque frame où l'objet apparaît
            for frame_id in tracked_objects[track_id]:
                x, y, w, h = tracked_objects[track_id][frame_id]
                
                ligne = f"{frame_id},{track_id},{x},{y},{w},{h},1,-1,-1,-1\n"
                f.write(ligne)
                
    print(f"Résultats sauvegardés dans {output_file}")

# Sauvegarder les résultats pour la séquence ADL-Rundle-6
sauvegarder_resultats(resultats_tracking, 'ADL-Rundle-6')


Résultats sauvegardés dans ADL-Rundle-6.txt


# TP3

In [146]:
class KalmanFilter:
    def __init__(self, dt, u_x, u_y, std_acc, x_std_meas, y_std_meas):
        self.dt = dt
        
        self.u = np.array([[u_x], [u_y]])
        
        self.x = np.array([[0], [0], [0], [0]])
        
        self.A = np.array([
            [1, 0, self.dt, 0],
            [0, 1, 0, self.dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        self.B = np.array([
            [(self.dt ** 2) / 2, 0],
            [0, (self.dt ** 2) / 2],
            [self.dt, 0],
            [0, self.dt]
        ])
        
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        
        self.Q = std_acc ** 2 * np.array([
            [(self.dt ** 4) / 4, 0, (self.dt ** 3) / 2, 0],
            [0, (self.dt ** 4) / 4, 0, (self.dt ** 3) / 2],
            [(self.dt ** 3) / 2, 0, self.dt ** 2, 0],
            [0, (self.dt ** 3) / 2, 0, self.dt ** 2]
        ])
        
        self.R = np.array([
            [x_std_meas ** 2, 0],
            [0, y_std_meas ** 2]
        ])
        
        self.P = np.eye(self.A.shape[1])

    def predict(self):
        self.x = np.dot(self.A, self.x) + np.dot(self.B, self.u)
        
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        
        return self.x[:2]

    def update(self, z):
        S = np.dot(np.dot(self.H, self.P), self.H.T) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))

        y = z - np.dot(self.H, self.x)
        self.x = self.x + np.dot(K, y)

        I = np.eye(self.H.shape[1])
        self.P = np.dot((I - np.dot(K, self.H)), self.P)

        return self.x[:2]

In [147]:
class Track:
    def __init__(self, bbox, track_id):
        self.id = track_id
        self.bbox = bbox
        self.frames_perdus = 0
        
        # Initialisation du centre
        cx = bbox[0] + bbox[2]/2
        cy = bbox[1] + bbox[3]/2
        
        # Paramètres Kalman plus conservateurs
        self.kalman = KalmanFilter(
            dt=1.0,
            u_x=0,
            u_y=0,
            std_acc=0.1,  # Réduit pour moins extrapoler le mouvement
            x_std_meas=0.1,
            y_std_meas=0.1
        )
        
        self.kalman.x = np.array([[cx], [cy], [0], [0]])
        self.derniere_detection = bbox
        self.vitesse_max = 50  # Vitesse maximale autorisée en pixels par frame
        
    def predict(self):
        if self.frames_perdus > 5:  # Si perdu trop longtemps, on arrête de prédire
            return self.derniere_detection
            
        predicted_center = self.kalman.predict()
        cx, cy = predicted_center.flatten()
        
        # Calculer le déplacement
        ancien_cx = self.bbox[0] + self.bbox[2]/2
        ancien_cy = self.bbox[1] + self.bbox[3]/2
        deplacement = np.sqrt((cx - ancien_cx)**2 + (cy - ancien_cy)**2)
        
        # Si le déplacement est trop grand, on limite la prédiction
        if deplacement > self.vitesse_max:
            ratio = self.vitesse_max / deplacement
            cx = ancien_cx + (cx - ancien_cx) * ratio
            cy = ancien_cy + (cy - ancien_cy) * ratio
        
        # Conserver la taille de la boîte
        w, h = self.bbox[2], self.bbox[3]
        nouvelle_bbox = [cx - w/2, cy - h/2, w, h]
        
        # Vérifier que la boîte reste dans des limites raisonnables
        if self.est_bbox_valide(nouvelle_bbox):
            self.bbox = nouvelle_bbox
        
        return self.bbox
    
    def update(self, detection):
        self.derniere_detection = detection
        self.bbox = detection
        self.frames_perdus = 0
        
        # Mise à jour Kalman avec le centre de la détection
        cx = detection[0] + detection[2]/2
        cy = detection[1] + detection[3]/2
        measurement = np.array([[cx], [cy]])
        self.kalman.update(measurement)
    
    def est_bbox_valide(self, bbox):
        # Vérifier que la boîte est dans des limites raisonnables
        x, y, w, h = bbox
        return (0 <= x <= 1920 and
                0 <= y <= 1080 and
                10 <= w <= 1000 and
                10 <= h <= 1000)

In [148]:
def trackerKalman(detections_par_frame):
    tracks = []
    next_track_id = 1
    resultats = {}
    active_tracks = set()  # Pour suivre les IDs actifs
    
    for frame_id, detections in detections_par_frame.items():
        # 1. Prédiction pour toutes les trajectoires existantes
        for track in tracks:
            track.predict()
        
        # 2. Association avec les nouvelles détections
        if tracks and len(detections) > 0:
            similarity_matrix = np.zeros((len(tracks), len(detections)))
            for i, track in enumerate(tracks):
                for j, detection in enumerate(detections):
                    # Vérifier si les boîtes se chevauchent significativement
                    iou = calculate_iou(track.bbox, detection)
                    similarity_matrix[i,j] = iou
            
            # Augmenter le seuil IoU pour éviter les associations ambiguës
            SEUIL_IOU = 0.5  # Seuil plus strict
            similarity_matrix[similarity_matrix < SEUIL_IOU] = 0
            
            # Ne permettre qu'une seule association par détection
            cost_matrix = 1 - similarity_matrix
            track_indices, det_indices = linear_sum_assignment(cost_matrix)
            
            matches = []
            unmatched_tracks = list(range(len(tracks)))
            unmatched_detections = list(range(len(detections)))
            
            # Filtrer les associations en vérifiant les chevauchements
            for track_idx, det_idx in zip(track_indices, det_indices):
                if similarity_matrix[track_idx, det_idx] >= SEUIL_IOU:
                    # Vérifier s'il n'y a pas déjà une trajectoire active dans cette zone
                    overlap_found = False
                    for other_track in tracks:
                        if other_track.id != tracks[track_idx].id and \
                           other_track.frames_perdus == 0 and \
                           calculate_iou(other_track.bbox, detections[det_idx]) > 0.3:
                            overlap_found = True
                            break
                    
                    if not overlap_found:
                        matches.append((track_idx, det_idx))
                        if track_idx in unmatched_tracks:
                            unmatched_tracks.remove(track_idx)
                        if det_idx in unmatched_detections:
                            unmatched_detections.remove(det_idx)
        else:
            matches = []
            unmatched_tracks = list(range(len(tracks)))
            unmatched_detections = list(range(len(detections)))
        
        # 3. Mise à jour des trajectoires
        for track_idx, det_idx in matches:
            tracks[track_idx].update(detections[det_idx])
            resultats.setdefault(tracks[track_idx].id, {})[frame_id] = tracks[track_idx].bbox
            active_tracks.add(tracks[track_idx].id)
        
        # 4. Gestion des trajectoires non associées
        MAX_FRAMES_PERDUES = 15  # Réduire pour supprimer plus rapidement les trajectoires perdues
        for track_idx in unmatched_tracks:
            tracks[track_idx].frames_perdus += 1
            if tracks[track_idx].frames_perdus <= MAX_FRAMES_PERDUES:
                resultats.setdefault(tracks[track_idx].id, {})[frame_id] = tracks[track_idx].bbox
            else:
                active_tracks.discard(tracks[track_idx].id)
        
        # 5. Supprimer les trajectoires perdues
        tracks = [track for track in tracks if track.frames_perdus <= MAX_FRAMES_PERDUES]
        
        # 6. Créer de nouvelles trajectoires
        MIN_DISTANCE = 50  # Distance minimale entre les centres des boîtes
        for det_idx in unmatched_detections:
            # Vérifier si la détection est suffisamment éloignée des trajectoires existantes
            detection = detections[det_idx]
            det_center = (detection[0] + detection[2]/2, detection[1] + detection[3]/2)
            
            trop_proche = False
            for track in tracks:
                track_box = track.bbox
                track_center = (track_box[0] + track_box[2]/2, track_box[1] + track_box[3]/2)
                distance = np.sqrt((det_center[0] - track_center[0])**2 + 
                                 (det_center[1] - track_center[1])**2)
                if distance < MIN_DISTANCE:
                    trop_proche = True
                    break
            
            if not trop_proche:
                new_track = Track(detection, next_track_id)
                tracks.append(new_track)
                resultats.setdefault(next_track_id, {})[frame_id] = detection
                active_tracks.add(next_track_id)
                next_track_id += 1
    
    # 7. Filtrer les trajectoires trop courtes
    MIN_DETECTIONS = 10  # Augmenter pour avoir des trajectoires plus stables
    trajectoires_filtrees = {}
    for track_id, track_data in resultats.items():
        if len(track_data) >= MIN_DETECTIONS:
            trajectoires_filtrees[track_id] = track_data
    
    return trajectoires_filtrees

In [149]:
detections_yolov5s_par_frame = {
    frame: det_yolov5s[det_yolov5s['frame'] == frame].iloc[:, 2:6].values.tolist()
    for frame in det_yolov5s['frame'].unique()
}

resultats_tracking = trackerKalman(detections_yolov5s_par_frame)

visualise_tracking_results('ADL-Rundle-6', resultats_tracking, 'tp3_result.mp4')